#### Robustness test (d+1) - Regression Analysis on Interest Rate Levels

This notebook performs robustness test on the regression analysis made on notebook 9. It does the same thing, but using the data from the interest rates one day after the publication of the minute instead.

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

In [2]:
futures_ir = pd.read_excel("./data/future_interest_rates/future_interest_rates.xlsx", index_col="Date")
futures_ir.index = pd.to_datetime(futures_ir.index)
futures_ir.sort_index(inplace=True)

#futures_ir_returns = futures_ir.pct_change().dropna()
futures_ir_returns = futures_ir.diff().dropna()

In [3]:
minutes_info = pd.read_excel("./data/raw/minutes_info.xlsx")
minutes_info = minutes_info[minutes_info["DataReferencia"] >= "2003-06-17"]
minutes_info = minutes_info.sort_values("DataReferencia", ascending=True)

dates_ref = minutes_info["DataReferencia"]
detes_pub = minutes_info["DataPublicacao"]

In [4]:
futures_ir_returns_d_plus_1 = futures_ir_returns.shift(-1) # IMPORTANT PART (D+1) -> THAT MEANS A -1 SHIFT
returns_pub_date_d_plus_1 = futures_ir_returns_d_plus_1[futures_ir_returns_d_plus_1.index.isin(detes_pub)]

#### Regression on optimism index

In [5]:
df_optimism = pd.read_excel("./data/processed/index_optimism.xlsx", index_col=0)
df_optimism = df_optimism.drop(columns=['filename'])
df_optimism.columns = [f"optimism_{i}" for i in range(len(df_optimism.columns))]

df_optimism = df_optimism.diff()

In [6]:
regression_data = pd.concat([returns_pub_date_d_plus_1, df_optimism], axis=1)
regression_data.dropna(inplace=True)

possible_y = returns_pub_date_d_plus_1.columns.tolist()
all_x = df_optimism.columns.tolist()

In [7]:
results_list = []

initial_X = regression_data[all_x]

for maturity in possible_y:
    Y = regression_data[maturity]
    X = initial_X.copy()

    p_value_threshold = 0.10

    while True:
        X = sm.add_constant(X)
        
        model = sm.OLS(Y, X).fit()

        pvalues = model.pvalues.drop('const')
        if pvalues.empty:
            break

        max_pvalue = pvalues.max()
        if max_pvalue > p_value_threshold:
            variable_to_remove = pvalues.idxmax()
            X = X.drop(variable_to_remove, axis=1)
        else:
            break
        
    if len(X.columns) > 0:
        results_list.append({
            'Vertix': maturity,
            'Betas': model.params[X.columns],
            'P-values': model.pvalues[X.columns],
            'R_sqrd_adj': model.rsquared_adj
        })

In [8]:
for i in results_list:
    print(i["Betas"])
    print(i["P-values"])
    print(i["R_sqrd_adj"])
    print("\n\n\n")

const         0.002560
optimism_0   -0.207177
optimism_2    0.163429
optimism_4   -0.327761
dtype: float64
const         0.039667
optimism_0    0.003406
optimism_2    0.038501
optimism_4    0.013710
dtype: float64
0.08806544773142522




const         0.004217
optimism_0   -0.316145
optimism_2    0.384505
dtype: float64
const         0.081881
optimism_0    0.020332
optimism_2    0.012549
dtype: float64
0.0445008593346482




const         0.006772
optimism_2    0.576212
dtype: float64
const         0.071302
optimism_2    0.015039
dtype: float64
0.026299634583061615




const         0.007786
optimism_2    1.075719
dtype: float64
const         0.223894
optimism_2    0.008006
dtype: float64
0.03219340883301092




const         0.006035
optimism_2    1.644758
dtype: float64
const         0.522885
optimism_2    0.006101
dtype: float64
0.03475049759754223






#### LASSO on Optimism index

In [9]:
results_list_lasso = []
X = regression_data[all_x]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

for maturity in possible_y:
    Y = regression_data[maturity]

    lasso_cv_model = LassoCV(cv=10, random_state=100, max_iter=10000)
    lasso_cv_model.fit(X_scaled, Y)

    selected_coeffs_mask = lasso_cv_model.coef_ != 0
    selected_variables_names = X.columns[selected_coeffs_mask].tolist()

    if selected_variables_names:
        X_selected_scaled = X_scaled[:, selected_coeffs_mask]
        X_selected_sm = sm.add_constant(X_selected_scaled)

        ols_final_model = sm.OLS(Y, X_selected_sm).fit()
        print(maturity)
        print(ols_final_model.summary())
        print("\n\n")

        ols_params = pd.Series(ols_final_model.params.values[1:], index=selected_variables_names)
        ols_pvalues = pd.Series(ols_final_model.pvalues.values[1:], index=selected_variables_names)

        results_list_lasso.append({
            'Vertix': maturity,
            'Alpha_Selecionado': lasso_cv_model.alpha_,
            'R_quadrado_LASSO': lasso_cv_model.score(X_scaled, Y),
            'Variáveis_Selecionadas_e_Coefs': dict(zip(selected_variables_names, lasso_cv_model.coef_[selected_coeffs_mask])),
            'Coeficientes_OLS_Final': ols_params,
            'P-valores_OLS_Final': ols_pvalues,
            'R_quadrado_Ajustado_OLS_Final': ols_final_model.rsquared_adj
        })
    else:
        results_list_lasso.append({
            'Vertix': maturity,
            'Alpha_Selecionado': lasso_cv_model.alpha_,
            'R_quadrado_LASSO': lasso_cv_model.score(X_scaled, Y),
            'Variáveis_Selecionadas_e_Coefs': {},
            'Coeficientes_OLS_Final': pd.Series(),
            'P-valores_OLS_Final': pd.Series(),
            'R_quadrado_Ajustado_OLS_Final': 0
        })

v_21
                            OLS Regression Results                            
Dep. Variable:                   v_21   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     4.924
Date:                Tue, 07 Oct 2025   Prob (F-statistic):           0.000299
Time:                        21:26:21   Log-Likelihood:                 501.61
No. Observations:                 187   AIC:                            -991.2
Df Residuals:                     181   BIC:                            -971.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0025      0.001      2.019    

In [10]:
for result in results_list_lasso:
    print(f"--- Resultados para a Maturidade (Vértice): {result['Vertix']} ---")
    print(f"Melhor alpha (λ) encontrado pela validação cruzada: {result['Alpha_Selecionado']:.6f}")
    print(f"R² do modelo LASSO: {result['R_quadrado_LASSO']:.4f}")
    print(f"R² Ajustado do modelo OLS final: {result['R_quadrado_Ajustado_OLS_Final']:.4f}")
    
    if not result['Variáveis_Selecionadas_e_Coefs']:
        print("Nenhuma variável foi selecionada pelo LASSO.")
    else:
        print("Variáveis selecionadas e seus coeficientes:")
        print("   Coeficientes do LASSO:")
        for var, coef in result['Variáveis_Selecionadas_e_Coefs'].items():
            print(f"     - {var}: {coef:.6f}")
        print("   Resultados do OLS final (para inferência):")
        for var in result['Coeficientes_OLS_Final'].index:
            coef = result['Coeficientes_OLS_Final'][var]
            pval = result['P-valores_OLS_Final'][var]
            print(f"     - {var}: Coeficiente = {coef:.6f}, P-valor = {pval:.4f}")
            
    print("-" * 50 + "\n")

--- Resultados para a Maturidade (Vértice): v_21 ---
Melhor alpha (λ) encontrado pela validação cruzada: 0.001436
R² do modelo LASSO: 0.0871
R² Ajustado do modelo OLS final: 0.0954
Variáveis selecionadas e seus coeficientes:
   Coeficientes do LASSO:
     - optimism_0: -0.002246
     - optimism_1: -0.000267
     - optimism_2: 0.001055
     - optimism_4: -0.001967
     - optimism_6: -0.000090
   Resultados do OLS final (para inferência):
     - optimism_0: Coeficiente = -0.003679, P-valor = 0.0037
     - optimism_1: Coeficiente = -0.001743, P-valor = 0.1624
     - optimism_2: Coeficiente = 0.002500, P-valor = 0.0465
     - optimism_4: Coeficiente = -0.003158, P-valor = 0.0124
     - optimism_6: Coeficiente = -0.001627, P-valor = 0.1899
--------------------------------------------------

--- Resultados para a Maturidade (Vértice): v_63 ---
Melhor alpha (λ) encontrado pela validação cruzada: 0.003872
R² do modelo LASSO: 0.0250
R² Ajustado do modelo OLS final: 0.0445
Variáveis selecionadas

#### Regression on optimism index

In [11]:
df_uncertainty = pd.read_excel("./data/processed/index_uncertainty.xlsx", index_col=0)
df_uncertainty = df_uncertainty.drop(columns=['filename'])
df_uncertainty.columns = [f"uncertainty_{i}" for i in range(len(df_uncertainty.columns))]

df_uncertainty = df_uncertainty.diff()

In [12]:
regression_data = pd.concat([returns_pub_date_d_plus_1, df_uncertainty], axis=1)
regression_data.dropna(inplace=True)

possible_y = returns_pub_date_d_plus_1.columns.tolist()
all_x = df_uncertainty.columns.tolist()

In [13]:
results_list = []

initial_X = regression_data[all_x]

for maturity in possible_y:
    Y = regression_data[maturity]
    X = initial_X.copy()

    p_value_threshold = 0.10

    while True:
        X = sm.add_constant(X)
        
        model = sm.OLS(Y, X).fit()

        pvalues = model.pvalues.drop('const')
        if pvalues.empty:
            break

        max_pvalue = pvalues.max()
        if max_pvalue > p_value_threshold:
            variable_to_remove = pvalues.idxmax()
            X = X.drop(variable_to_remove, axis=1)
        else:
            break
        
    if len(X.columns) > 0:
        results_list.append({
            'Vertix': maturity,
            'Betas': model.params[X.columns],
            'P-values': model.pvalues[X.columns],
            'R_sqrd_adj': model.rsquared_adj
        })

In [14]:
for i in results_list:
    print(i["Betas"])
    print(i["P-values"])
    print(i["R_sqrd_adj"])
    print("\n\n\n")

const            0.002483
uncertainty_4   -0.323601
uncertainty_5    0.168641
dtype: float64
const            0.052990
uncertainty_4    0.014530
uncertainty_5    0.031392
dtype: float64
0.0286037658156002




const            0.004144
uncertainty_3    0.476893
dtype: float64
const            0.091747
uncertainty_3    0.042238
dtype: float64
0.016827070315392878




const            0.006692
uncertainty_2    0.666654
uncertainty_4   -1.253547
uncertainty_5    0.480219
dtype: float64
const            0.071006
uncertainty_2    0.055550
uncertainty_4    0.002471
uncertainty_5    0.040749
dtype: float64
0.051145020663670326




const            0.007642
uncertainty_2    1.850756
uncertainty_4   -1.666460
dtype: float64
const            0.228534
uncertainty_2    0.001463
uncertainty_4    0.013357
dtype: float64
0.04868152473720677




const            0.005800
uncertainty_2    1.924191
uncertainty_4   -1.974913
dtype: float64
const            0.542022
uncertainty_2    0.026416
uncertainty_4 

#### LASSO on Uncertainty

In [15]:
results_list_lasso = []
X = regression_data[all_x]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

for maturity in possible_y:
    Y = regression_data[maturity]

    lasso_cv_model = LassoCV(cv=10, random_state=100, max_iter=10000)
    lasso_cv_model.fit(X_scaled, Y)

    selected_coeffs_mask = lasso_cv_model.coef_ != 0
    selected_variables_names = X.columns[selected_coeffs_mask].tolist()

    if selected_variables_names:
        X_selected_scaled = X_scaled[:, selected_coeffs_mask]
        X_selected_sm = sm.add_constant(X_selected_scaled)

        ols_final_model = sm.OLS(Y, X_selected_sm).fit()
        print(maturity)
        print(ols_final_model.summary())
        print("\n\n")
        
        ols_params = pd.Series(ols_final_model.params.values[1:], index=selected_variables_names)
        ols_pvalues = pd.Series(ols_final_model.pvalues.values[1:], index=selected_variables_names)

        results_list_lasso.append({
            'Vertix': maturity,
            'Alpha_Selecionado': lasso_cv_model.alpha_,
            'R_quadrado_LASSO': lasso_cv_model.score(X_scaled, Y),
            'Variáveis_Selecionadas_e_Coefs': dict(zip(selected_variables_names, lasso_cv_model.coef_[selected_coeffs_mask])),
            'Coeficientes_OLS_Final': ols_params,
            'P-valores_OLS_Final': ols_pvalues,
            'R_quadrado_Ajustado_OLS_Final': ols_final_model.rsquared_adj
        })
    else:
        results_list_lasso.append({
            'Vertix': maturity,
            'Alpha_Selecionado': lasso_cv_model.alpha_,
            'R_quadrado_LASSO': lasso_cv_model.score(X_scaled, Y),
            'Variáveis_Selecionadas_e_Coefs': {},
            'Coeficientes_OLS_Final': pd.Series(),
            'P-valores_OLS_Final': pd.Series(),
            'R_quadrado_Ajustado_OLS_Final': 0
        })



v_21
                            OLS Regression Results                            
Dep. Variable:                   v_21   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     2.719
Date:                Tue, 07 Oct 2025   Prob (F-statistic):              0.101
Time:                        21:26:28   Log-Likelihood:                 491.05
No. Observations:                 187   AIC:                            -978.1
Df Residuals:                     185   BIC:                            -971.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0025      0.001      1.929    

In [16]:
for result in results_list_lasso:
    print(f"--- Resultados para a Maturidade (Vértice): {result['Vertix']} ---")
    print(f"Melhor alpha (λ) encontrado pela validação cruzada: {result['Alpha_Selecionado']:.6f}")
    print(f"R² do modelo LASSO: {result['R_quadrado_LASSO']:.4f}")
    print(f"R² Ajustado do modelo OLS final: {result['R_quadrado_Ajustado_OLS_Final']:.4f}")
    
    if not result['Variáveis_Selecionadas_e_Coefs']:
        print("Nenhuma variável foi selecionada pelo LASSO.")
    else:
        print("Variáveis selecionadas e seus coeficientes:")
        print("   Coeficientes do LASSO:")
        for var, coef in result['Variáveis_Selecionadas_e_Coefs'].items():
            print(f"     - {var}: {coef:.6f}")
        print("   Resultados do OLS final (para inferência):")
        for var in result['Coeficientes_OLS_Final'].index:
            coef = result['Coeficientes_OLS_Final'][var]
            pval = result['P-valores_OLS_Final'][var]
            print(f"     - {var}: Coeficiente = {coef:.6f}, P-valor = {pval:.4f}")
            
    print("-" * 50 + "\n")

--- Resultados para a Maturidade (Vértice): v_21 ---
Melhor alpha (λ) encontrado pela validação cruzada: 0.002123
R² do modelo LASSO: 0.0000
R² Ajustado do modelo OLS final: 0.0092
Variáveis selecionadas e seus coeficientes:
   Coeficientes do LASSO:
     - uncertainty_4: -0.000000
   Resultados do OLS final (para inferência):
     - uncertainty_4: Coeficiente = -0.002123, P-valor = 0.1008
--------------------------------------------------

--- Resultados para a Maturidade (Vértice): v_63 ---
Melhor alpha (λ) encontrado pela validação cruzada: 0.004056
R² do modelo LASSO: 0.0076
R² Ajustado do modelo OLS final: 0.0168
Variáveis selecionadas e seus coeficientes:
   Coeficientes do LASSO:
     - uncertainty_3: 0.000944
   Resultados do OLS final (para inferência):
     - uncertainty_3: Coeficiente = 0.005001, P-valor = 0.0422
--------------------------------------------------

--- Resultados para a Maturidade (Vértice): v_126 ---
Melhor alpha (λ) encontrado pela validação cruzada: 0.0031